In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import scipy.io
import glob
import os
from math import sqrt

In [2]:
def get_distance_2_pts(p1,p2):
    x1,y1 = p1
    x2,y2 = p2
    distance = sqrt((y1-y2)**2+(x2-x1)**2)
    return distance

In [3]:
# w is the length of the rectangle
def get_4_pts(p1,p2,img):
    x_max,y_max,_ = img.shape
    x1,y1 = p1
    x2,y2 = p2
    alpha = sqrt((y1-y2)**2+(x2-x1)**2)
    
    if alpha > 200 :
        w = -125
    else:
        w = -250
    
    sin_theta = (y1-y2)/alpha
    cos_theta = (x2-x1)/alpha
    x4 = w*sin_theta + x2
    x3 = w*sin_theta + x1
    y4 = w*cos_theta + y2
    y3 = w*cos_theta + y1
    if (x4 > x_max):
        x4 = x_max
    if (x3 > x_max):
        x3 = x_max
    if (y4 > y_max):
        y4 = y_max
    if (y3 > y_max):
        y3 = y_max
    if (x4 < 0):
        x4 = 0
    if (x3 < 0):
        x3 = 0
    if (y4 < 0):
        y4 = 0
    if (y3 < 0):
        y3 = 0
    p3 = [x3,y3]
    p4 = [x4,y4]
    return p1,p2,p3,p4

In [4]:
def get_slots(img,mat):
    warped_images_with_vacancy = []
    for s in mat['slots']:
        #get the four points
        p1_num = s[0] - 1
        p2_num = s[1] - 1
        p1 = mat['marks'][p1_num]
        p2 = mat['marks'][p2_num]
        pts = get_4_pts(p1,p2,img)
        _,_,p3,p4 = pts
        #get vacancy
        vacancy_num = s[2]
        vacancy = 'non-vacant'
        if (vacancy_num == 0):
            vacancy = 'vacant'
        elif (vacancy_num == 1):
            vacancy = 'non-vacant'
        elif (vacancy_num == 2):
            vacancy = 'parking'
        #get slot only image(s)
        pts_src = np.array([p1,p2,p3,p4],np.float32)    
        width = get_distance_2_pts(p1,p2)
        height = get_distance_2_pts(p4,p2)
        pts_dst = np.array([[0.0,0.0],[width, 0.0],[ 0.0,height],[width,height]],np.float32)
        m_warp = cv2.getPerspectiveTransform(pts_src, pts_dst)
        warp_img = cv2.warpPerspective(img, m_warp, (int(width), int(height)))
        warped_images_with_vacancy.append([warp_img,vacancy])
    return warped_images_with_vacancy

In [5]:
def img_resize(img):
    #if x >> y rotate 90 degress
    if (img.shape[0]/img.shape[1]) >= 1.3 :
        img =  cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
    img = cv2.resize(img, (120,46))
    return img

In [6]:
iterator = 0
num_samples = -1
training_dir = 'training/'
new_data_set_dir = 'training_vacancy/'
for file in glob.glob(training_dir + '*.jpg'):
    file_name = file[:-4]
    img = cv2.imread(file)
    mat = scipy.io.loadmat(file_name + '.mat')
    warped_images_with_vacancy = get_slots(img,mat)
    for image,vacancy in warped_images_with_vacancy:
        slot_name = 'training' + str(iterator)
        image = img_resize(image)
        cv2.imwrite(new_data_set_dir+vacancy+'/'+slot_name+'.jpg',image)
    iterator += 1
    #if iterator >= num_samples:
    #    break